### Ingestion de la carpeta "movie_company"


In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

### Leer los archivos CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
movies_companies_schema = StructType(fields=[
     StructField("movieId", IntegerType(), True),
     StructField("companyId", StringType(), True),
])

In [0]:
movies_companies_df = spark.read \
                   .schema(movies_companies_schema) \
                   .csv(f"{bronze_folder_path}/{v_file_date}/movie_company") 

### Paso 2 - Renombrar las columnas y añadir columnas nuevas
1. "movieId" renombrar a "movie_id"
2. "companyId" renombrar a "company_id"
3.  Agregar las columnas de "ingestion_date" y "envoronment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movies_companies_final_df = add_ingestion_date(movies_companies_df) \
                            .withColumn("environment", lit(v_environment)) \
                            .withColumn("file_date", lit(v_file_date)) \
                            .withColumnsRenamed(
                                {"movieId": "movie_id", 
                                "companyId": "company_id"
                                })
                         

### Paso 3 - Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(movies_companies_final_df,"movie_silver", "movies_companies","file_date")

In [0]:
# movies_companies_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/movies_companies")

# Crear una tabla en base al data frame movies_companies_final_df
# Creara la tabla movies_companies en a DB que se llame movie_silver
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Silver

#movies_companies_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.movies_companies")

# Append, agrega los registros a la tabla existente, carga incremental de datos
#movies_companies_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movies_companies")

#Modulo: Delta Lake
merge_condition = "tgt.movie_id = src.movie_id AND tgt.company_id = src.company_id AND tgt.file_date = src.file_date" # Buscar por particion
merge_delta_lake(movies_companies_final_df,"movie_silver","movies_companies",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
-- Consultar la DB movie_silver, tabla movies_companies
SELECT file_date, COUNT(1)
FROM movie_silver.movies_companies
GROUP BY file_date;

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/movies_companies"))
display(spark.read.format("delta").load(f"{silver_folder_path}/movies_companies"))

In [0]:
dbutils.notebook.exit("Success")